In [ ]:
from  utils import *
endpoint = "http://192.168.116.131:8181/blazegraph/namespace/kb/sparql"


In [ ]:
classesQuery = """
PREFIX sch:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX syn:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
SELECT * WHERE{
   ?profile sch:label "Example_Вывод из эксплуатации участка линии".
   ?profile syn:type owl:Ontology.
   ?class sch:isDefinedBy ?profile.
   ?class syn:type 	owl:Class.
   OPTIONAL{?class sch:subClassOf ?parent}
}
"""

In [ ]:
fieldsQuery = """
PREFIX sch:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX syn:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX ext:  <http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#>
SELECT * WHERE{
   ?profile sch:label "Example_Вывод из эксплуатации участка линии".
   ?profile syn:type owl:Ontology.
   ?class sch:isDefinedBy ?profile.
   ?class syn:type owl:Class.
   ?field sch:domain ?class.
   ?field syn:type ?type.
   OPTIONAL{?field sch:range ?range.}
   OPTIONAL{?field ext:multiplicity ?multiplicity.}
}
"""

Выполнение запросов

In [ ]:
classesDF = sparqlDF(endpoint, classesQuery, True)
fieldsDF = sparqlDF(endpoint, fieldsQuery, True)
classes = dataFrameToObjects(classesDF)
fields = dataFrameToObjects(fieldsDF)

In [ ]:


concreteClases = list(filter(lambda c1: not any(
    c2["parent"] == c1["class"] for c2 in classes), classes))
mainClasses = list(filter(lambda c: not any(
    f["range"] == c["class"] and f["multiplicity"][-1] == "n" for f in fields), concreteClases))


def collectClassFields(leaf, cur):
    for field in filter(lambda f: f["class"] == cur["class"], fields):
        field["refClass"] = None
        if (field["type"] == "ObjectProperty" and field["multiplicity"][-1] == "n"):
            field["refClass"] = next(
                (c for c in concreteClases if c["class"] == field["range"]), None)
        leaf["fields"].append(field)

    if (cur["parent"]):
        nxt = next((c for c in classes if c["class"] == cur["parent"]), None)
        collectClassFields(leaf, nxt)


for c in concreteClases:
    c["fields"] = []
    collectClassFields(c, c)


# def testPrint(c, pref):
#     if (pref == ""):
#         print(c["class"])
#     pref += "---"
#     for f in c["fields"]:
#         print(pref, f["field"], ":", f["range"])
#         if (f["refClass"]):
#             testPrint(f["refClass"], pref)


# for c in mainClasses:
#     testPrint(c, "")


In [17]:
classesDF.head(1000)

,profile,class,parent
0,Example_ВыводИзЭксплуатацииУчасткаЛинии,IdentifiedObject,None
1,Example_ВыводИзЭксплуатацииУчасткаЛинии,AccountPartLine,IdentifiedObject
2,Example_ВыводИзЭксплуатацииУчасткаЛинии,PowerSystemResource,IdentifiedObject
3,Example_ВыводИзЭксплуатацииУчасткаЛинии,ConnectivityNodeContainer,PowerSystemResource
4,Example_ВыводИзЭксплуатацииУчасткаЛинии,EquipmentContainer,ConnectivityNodeContainer
5,Example_ВыводИзЭксплуатацииУчасткаЛинии,Line,EquipmentContainer


In [ ]:
fieldsDF.head(1000)